# `polars_bloomberg` Examples
(c) 2024 Marek Ozana

In [1]:
# Imports
from datetime import date

from polars_bloomberg import BQuery

## Bloomberg Data Point
API:
```python
bdp(securities: List[str],
    fields: List[str],
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)
```

In [2]:
# Bloomberg Data Point
with BQuery() as bq:
    df = bq.bdp(["AAPL US Equity"], ["PX_LAST"])
    print(df)

shape: (1, 2)
┌────────────────┬─────────┐
│ security       ┆ PX_LAST │
│ ---            ┆ ---     │
│ str            ┆ f64     │
╞════════════════╪═════════╡
│ AAPL US Equity ┆ 248.13  │
└────────────────┴─────────┘


In [4]:
with BQuery() as bq:
    df = bq.bdp(["AAPL US Equity", "SEBA SS Equity"], ["PX_LAST", "CRNCY"])
    print(df)

shape: (2, 3)
┌────────────────┬─────────┬───────┐
│ security       ┆ PX_LAST ┆ CRNCY │
│ ---            ┆ ---     ┆ ---   │
│ str            ┆ f64     ┆ str   │
╞════════════════╪═════════╪═══════╡
│ AAPL US Equity ┆ 248.13  ┆ USD   │
│ SEBA SS Equity ┆ 155.2   ┆ SEK   │
└────────────────┴─────────┴───────┘


In [5]:
with BQuery() as bq:
    df = bq.bdp(
        ["XS2930103580 Corp", "USX60003AC87 Corp"],
        ["SECURITY_DES", "YAS_ZSPREAD", "CRNCY", "NXT_CALL_DT"],
    )
    print(df)

shape: (2, 5)
┌───────────────────┬────────────────┬─────────────┬───────┬─────────────┐
│ security          ┆ SECURITY_DES   ┆ YAS_ZSPREAD ┆ CRNCY ┆ NXT_CALL_DT │
│ ---               ┆ ---            ┆ ---         ┆ ---   ┆ ---         │
│ str               ┆ str            ┆ f64         ┆ str   ┆ date        │
╞═══════════════════╪════════════════╪═════════════╪═══════╪═════════════╡
│ XS2930103580 Corp ┆ SEB 6 3/4 PERP ┆ 304.676112  ┆ USD   ┆ 2031-11-04  │
│ USX60003AC87 Corp ┆ NDAFH 6.3 PERP ┆ 292.477506  ┆ USD   ┆ 2031-09-25  │
└───────────────────┴────────────────┴─────────────┴───────┴─────────────┘


In [7]:
with BQuery() as bq:
    df = bq.bdp(
        ["IBM US Equity"],
        ["PX_LAST", "CRNCY_ADJ_PX_LAST"],
        overrides=[("EQY_FUND_CRNCY", "SEK")],
    )
    print(df)

shape: (1, 3)
┌───────────────┬─────────┬───────────────────┐
│ security      ┆ PX_LAST ┆ CRNCY_ADJ_PX_LAST │
│ ---           ┆ ---     ┆ ---               │
│ str           ┆ f64     ┆ f64               │
╞═══════════════╪═════════╪═══════════════════╡
│ IBM US Equity ┆ 230.82  ┆ 2535.174          │
└───────────────┴─────────┴───────────────────┘


In [8]:
# Override string dates in format YYYYMMDD
with BQuery() as bq:
    df = bq.bdp(
        ["USX60003AC87 Corp"], ["SETTLE_DT"],
        overrides=[("USER_LOCAL_TRADE_DATE", "20241014")],
    )
    print(df)

shape: (1, 2)
┌───────────────────┬────────────┐
│ security          ┆ SETTLE_DT  │
│ ---               ┆ ---        │
│ str               ┆ date       │
╞═══════════════════╪════════════╡
│ USX60003AC87 Corp ┆ 2024-10-15 │
└───────────────────┴────────────┘


In [9]:
# Another override example
with BQuery() as bq:
    df = bq.bdp(
        ["USDSEK Curncy", "SEKCZK Curncy"],
        ["SETTLE_DT", "PX_LAST"],
        overrides=[("REFERENCE_DATE", "20200715")],
    )
    print(df)

shape: (2, 3)
┌───────────────┬────────────┬─────────┐
│ security      ┆ SETTLE_DT  ┆ PX_LAST │
│ ---           ┆ ---        ┆ ---     │
│ str           ┆ date       ┆ f64     │
╞═══════════════╪════════════╪═════════╡
│ USDSEK Curncy ┆ 2020-07-17 ┆ 10.9778 │
│ SEKCZK Curncy ┆ 2020-07-17 ┆ 2.1698  │
└───────────────┴────────────┴─────────┘


## Bloomberg Data History
```python
bdh(securities: List[str],
    fields: List[str],
    start_date: date,
    end_date: date,
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)

```

In [8]:
# Bloomberg Data History
with BQuery() as bq:
    df = bq.bdh(
        ["SPY US Equity", "TLT US Equity"],
        ["PX_LAST", "VOLUME"],
        start_date=date(2019, 1, 1),
        end_date=date(2019, 1, 10),
        options={"adjustmentSplit": True},
    )
df.tail(5)

security,date,PX_LAST,VOLUME
str,date,f64,f64
"""TLT US Equity""",2019-01-04,122.11,1.2970226e7
"""TLT US Equity""",2019-01-07,121.75,8.498104e6
"""TLT US Equity""",2019-01-08,121.43,7.737103e6
"""TLT US Equity""",2019-01-09,121.24,9.349245e6
"""TLT US Equity""",2019-01-10,120.46,8.22286e6


In [ ]:
with BQuery() as bq:
    df = bq.bdh(
        ["AAPL US Equity"],
        ["PX_LAST"],
        start_date=date(2019, 1, 1),
        end_date=date(2019, 3, 29),
        options={"periodicitySelection": "MONTHLY"},
    )
df

security,date,PX_LAST
str,date,f64
"""AAPL US Equity""",2019-01-31,41.61
"""AAPL US Equity""",2019-02-28,43.288
"""AAPL US Equity""",2019-03-29,47.488


## Bloomberg Query Language
```python
bql(expression: str)
```

In [ ]:
with BQuery() as bq:
    df = bq.bql("get(px_last) for(['IBM US Equity', 'OMX Index'])")
df

shape: (2, 4)
┌───────────────┬────────────┬──────────────┬──────────────────┐
│ ID            ┆ px_last    ┆ px_last.DATE ┆ px_last.CURRENCY │
│ ---           ┆ ---        ┆ ---          ┆ ---              │
│ str           ┆ f64        ┆ date         ┆ str              │
╞═══════════════╪════════════╪══════════════╪══════════════════╡
│ IBM US Equity ┆ 231.024994 ┆ 2024-12-10   ┆ USD              │
│ OMX Index     ┆ 2602.806   ┆ 2024-12-10   ┆ SEK              │
└───────────────┴────────────┴──────────────┴──────────────────┘


In [ ]:
# Example: average PE ratio per sector
query = """
let(#avg_pe=avg(group(pe_ratio(), gics_sector_name()));)
get(#avg_pe)
for(members('SPX Index'))
"""
with BQuery() as bq:
    df = bq.bql(query)
df

ID,#avg_pe,#avg_pe.REVISION_DATE,#avg_pe.AS_OF_DATE,#avg_pe.PERIOD_END_DATE,#avg_pe.ORIG_IDS,#avg_pe.GICS_SECTOR_NAME()
str,f64,date,date,date,str,str
"""Communication Services""",26.418942,2024-11-14,2024-12-10,2024-09-30,null,"""Communication Services"""
"""Consumer Discretionary""",26.388028,2024-12-10,2024-12-10,2024-11-23,null,"""Consumer Discretionary"""
"""Consumer Staples""",19.591445,2024-12-06,2024-12-10,2024-11-09,null,"""Consumer Staples"""
"""Energy""",18.772598,2024-11-12,2024-12-10,2024-09-30,null,"""Energy"""
"""Financials""",37.344488,2024-11-26,2024-12-10,2024-09-30,null,"""Financials"""
…,…,…,…,…,…,…
"""Industrials""",37.453192,2024-11-26,2024-12-10,2024-10-31,null,"""Industrials"""
"""Information Technology""",57.043857,2024-12-09,2024-12-10,2024-11-30,null,"""Information Technology"""
"""Materials""",24.861414,2024-11-21,2024-12-10,2024-09-30,null,"""Materials"""


In [ ]:
# Example: Duration and ZSpread for search results
query="""
let(#dur=duration(duration_type=MODIFIED);
    #zsprd=spread(spread_type=Z);)
get(name(), #dur, #zsprd)
for(filter(screenresults(type=SRCH, screen_name='@COCO'),
           ticker in ['SEB', 'SHBASS']))
"""

with BQuery() as bq:
    df = bq.bql(query)
df

shape: (6, 6)
┌───────────────┬─────────────────┬──────────┬────────────┬────────────┬─────────────┐
│ ID            ┆ name()          ┆ #dur     ┆ #dur.DATE  ┆ #zsprd     ┆ #zsprd.DATE │
│ ---           ┆ ---             ┆ ---      ┆ ---        ┆ ---        ┆ ---         │
│ str           ┆ str             ┆ f64      ┆ date       ┆ f64        ┆ date        │
╞═══════════════╪═════════════════╪══════════╪════════════╪════════════╪═════════════╡
│ BW924993 Corp ┆ SEB 6 ⅞ PERP    ┆ 2.244382 ┆ 2024-12-10 ┆ 229.930933 ┆ 2024-12-10  │
│ ZO703956 Corp ┆ SHBASS 4 ¾ PERP ┆ 4.958582 ┆ 2024-12-10 ┆ 269.963777 ┆ 2024-12-10  │
│ ZO703315 Corp ┆ SHBASS 4 ⅜ PERP ┆ 1.968658 ┆ 2024-12-10 ┆ 232.839648 ┆ 2024-12-10  │
│ YU819930 Corp ┆ SEB 6 ¾ PERP    ┆ 5.388785 ┆ 2024-12-10 ┆ 324.70196  ┆ 2024-12-10  │
│ ZQ349286 Corp ┆ SEB 5 ⅛ PERP    ┆ 0.409083 ┆ 2024-12-10 ┆ 165.405465 ┆ 2024-12-10  │
│ YV402592 Corp ┆ SEB Float PERP  ┆ 0.22527  ┆ 2024-12-10 ┆ 248.756    ┆ 2024-12-10  │
└───────────────┴────────────

In [ ]:
# Example: Average spread for an issuer
query = """
let(
    #bins = bins(maturity_years,
                 [3,9,18,30],
                 ['(1) 0-3','(2) 3-9','(3) 9-18','(4) 18-30','(5) 30+']);
    #average_spread = avg(group(spread(st=oas),#bins));
)
get(#average_spread)
for(filter(bonds('NVDA US Equity', issuedby = 'ENTITY'),
           maturity_years != NA))
"""

with BQuery() as bq:
    df = bq.bql(query)
df.head(5)

shape: (5, 5)
┌───────────┬─────────────────┬──────────────────────┬──────────────────────┬──────────────────────┐
│ ID        ┆ #average_spread ┆ #average_spread.DATE ┆ #average_spread.ORIG ┆ #average_spread.#BIN │
│ ---       ┆ ---             ┆ ---                  ┆ _IDS                 ┆ S                    │
│ str       ┆ f64             ┆ date                 ┆ ---                  ┆ ---                  │
│           ┆                 ┆                      ┆ str                  ┆ str                  │
╞═══════════╪═════════════════╪══════════════════════╪══════════════════════╪══════════════════════╡
│ (1) 0-3   ┆ 30.638311       ┆ 2024-12-10           ┆ QZ552396 Corp        ┆ (1) 0-3              │
│ (2) 3-9   ┆ 59.772151       ┆ 2024-12-10           ┆ null                 ┆ (2) 3-9              │
│ (3) 9-18  ┆ 106.722341      ┆ 2024-12-10           ┆ BH393780 Corp        ┆ (3) 9-18             │
│ (4) 18-30 ┆ 129.945414      ┆ 2024-12-10           ┆ BH393781 Corp        ┆

alt.Chart(...)

In [ ]:
# stocks with 20d EMA above 200d EMA and RSI > 70
with BQuery() as bq:
    df = bq.bql(
        """
        let(#ema20=emavg(period=20);
            #ema200=emavg(period=200);
            #rsi=rsi(close=px_last());)
        get(name(), #ema20, #ema200, #rsi)
        for(filter(members('OMX Index'),
                    and(#ema20 > #ema200, #rsi > 70)))
        with(fill=PREV)
        """
    )
df

ID,name(),#ema20,#ema20.DATE,#ema20.CURRENCY,#ema200,#ema200.DATE,#ema200.CURRENCY,#rsi,#rsi.DATE
str,str,f64,date,str,f64,date,str,f64,date
"""VOLVB SS Equity""","""Volvo AB""",278.632944,2024-12-10,"""SEK""",268.314068,2024-12-10,"""SEK""",71.328102,2024-12-10
"""SKFB SS Equity""","""SKF AB""",212.787284,2024-12-10,"""SEK""",204.591927,2024-12-10,"""SEK""",75.07462,2024-12-10


In [ ]:
query="""
let(#ax=axes();)
get(ticker, cpn(), nxt_call_dt(), #ax)
for(filter(bondsuniv(ACTIVE),
    crncy()=='USD' and
    basel_iii_designation() == 'Additional Tier 1' and
    country_iso() == 'SE' and
    is_axed('Bid') == True))
"""

with BQuery() as bq:
    df = bq.bql(query)
df.head()

ID,ticker,cpn(),cpn().MULTIPLIER,cpn().CPN_TYP,nxt_call_dt(),#ax,#ax.ASK_DEPTH,#ax.BID_DEPTH,#ax.ASK_TOTAL_SIZE,#ax.BID_TOTAL_SIZE
str,str,f64,f64,str,date,str,i64,i64,f64,f64
"""YU819930 Corp""","""SEB""",6.75,1.0,"""VARIABLE""",2031-11-04,"""Y""",2,3,6e6,3.8e6
"""ZF859199 Corp""","""SWEDA""",7.75,1.0,"""VARIABLE""",2030-03-17,"""Y""",2,3,7e6,1.1e7
"""ZL122341 Corp""","""SWEDA""",7.625,1.0,"""VARIABLE""",2028-03-17,"""Y""",1,8,2e6,3.34e7
"""ZO703315 Corp""","""SHBASS""",4.375,1.0,"""VARIABLE""",2027-03-01,"""Y""",null,3,null,1.14e7
"""ZQ349286 Corp""","""SEB""",5.125,1.0,"""VARIABLE""",2025-05-13,"""Y""",3,9,6.7e6,5.02e7


In [ ]:
# Get Bond Universe from Equity Ticker
query = """
let(#rank=normalized_payment_rank();
    #oas=spread(st=oas);
    #nxt_call=nxt_call_dt();
    )
get(name(), #rank, #nxt_call, #oas)
for(filter(bonds('GTN US Equity'), series() == '144A'))
"""

with BQuery() as bq:
    df = bq.bql(query)
df

shape: (5, 6)
┌───────────────┬───────────────────┬──────────────────┬────────────┬─────────────┬────────────┐
│ ID            ┆ name()            ┆ #rank            ┆ #nxt_call  ┆ #oas        ┆ #oas.DATE  │
│ ---           ┆ ---               ┆ ---              ┆ ---        ┆ ---         ┆ ---        │
│ str           ┆ str               ┆ str              ┆ date       ┆ f64         ┆ date       │
╞═══════════════╪═══════════════════╪══════════════════╪════════════╪═════════════╪════════════╡
│ YX231113 Corp ┆ GTN 10 ½ 07/15/29 ┆ 1st Lien Secured ┆ 2026-07-15 ┆ 615.798149  ┆ 2024-12-10 │
│ BS116983 Corp ┆ GTN 5 ⅜ 11/15/31  ┆ Sr Unsecured     ┆ 2026-11-15 ┆ 1144.393892 ┆ 2024-12-10 │
│ AV438089 Corp ┆ GTN 7 05/15/27    ┆ Sr Unsecured     ┆ 2024-12-17 ┆ 389.022271  ┆ 2024-12-10 │
│ ZO860846 Corp ┆ GTN 4 ¾ 10/15/30  ┆ Sr Unsecured     ┆ 2025-10-15 ┆ 1184.969597 ┆ 2024-12-10 │
│ LW375188 Corp ┆ GTN 5 ⅞ 07/15/26  ┆ Sr Unsecured     ┆ 2025-01-06 ┆ 185.544312  ┆ 2024-12-10 │
└───────────────

alt.LayerChart(...)

In [ ]:
# Total Return of GTN Bonds
query="""
let(#rng = range(-3M, 0D);
    #rets = return_series(calc_interval=#rng,per=W);
    )
get(#rets)
for(filter(bonds('GTN US Equity'), series() == '144A'))
"""

with BQuery() as bq:
    df = bq.bql(query)
df


shape: (20, 3)
┌───────────────┬───────────┬────────────┐
│ ID            ┆ #rets     ┆ #rets.DATE │
│ ---           ┆ ---       ┆ ---        │
│ str           ┆ f64       ┆ date       │
╞═══════════════╪═══════════╪════════════╡
│ YX231113 Corp ┆ null      ┆ 2024-11-19 │
│ YX231113 Corp ┆ 0.005028  ┆ 2024-11-26 │
│ YX231113 Corp ┆ 0.000326  ┆ 2024-12-03 │
│ YX231113 Corp ┆ 0.000414  ┆ 2024-12-10 │
│ BS116983 Corp ┆ null      ┆ 2024-11-19 │
│ …             ┆ …         ┆ …          │
│ ZO860846 Corp ┆ -0.010198 ┆ 2024-12-10 │
│ LW375188 Corp ┆ null      ┆ 2024-11-19 │
│ LW375188 Corp ┆ -0.000997 ┆ 2024-11-26 │
│ LW375188 Corp ┆ 0.001294  ┆ 2024-12-03 │
│ LW375188 Corp ┆ 0.0011    ┆ 2024-12-10 │
└───────────────┴───────────┴────────────┘


In [ ]:
# TODO: Fix this query
query="""
    let(#segment=segment_name();
        #revenue=sales_Rev_turn(fpt=q, fpr=range(2023Q3, 2024Q3));
        )
    get(#segment, #revenue)
    for(segments('GTN US Equity',type=reported,hierarchy=PRODUCT, level=1))
"""
with BQuery() as bq:
    df = bq.bql(query)
df